In [1]:
import os
os.environ['PYSPARK_SUBMIT_ARGS'] = '--packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.0.2 pyspark-shell'

In [2]:
import findspark
findspark.init('/usr/local/spark')
# import pyspark

Import StreamingContext which is the main entry point for all streaming functionality.

In [3]:
from pyspark import SparkContext

In [4]:
from pyspark.streaming import StreamingContext

In [5]:
from pyspark.streaming.kafka import KafkaUtils

Create a SparkContext with two execution threads, and StreamingContext with batch interval of 1 second.

In [6]:
sc = SparkContext("local[2]", "StreamingKafka2")
ssc = StreamingContext(sc, 1)

Create an input DStream using KafkaUtils.createDirectStream passing the parameters – Spark Streaming Context, topic and broker(s).

In [7]:
kafkaStream = KafkaUtils.createDirectStream(ssc, ['kafka-streaming2'], {"metadata.broker.list": 'localhost:9092'})

Py4JJavaError: An error occurred while calling o29.createDirectStreamWithoutMessageHandler.
: org.apache.spark.SparkException: org.apache.spark.SparkException: Error getting partition metadata for 'kafka-streaming2'. Does the topic exist?
	at org.apache.spark.streaming.kafka.KafkaCluster$$anonfun$checkErrors$1.apply(KafkaCluster.scala:373)
	at org.apache.spark.streaming.kafka.KafkaCluster$$anonfun$checkErrors$1.apply(KafkaCluster.scala:373)
	at scala.util.Either.fold(Either.scala:98)
	at org.apache.spark.streaming.kafka.KafkaCluster$.checkErrors(KafkaCluster.scala:372)
	at org.apache.spark.streaming.kafka.KafkaUtils$.getFromOffsets(KafkaUtils.scala:222)
	at org.apache.spark.streaming.kafka.KafkaUtilsPythonHelper.createDirectStream(KafkaUtils.scala:720)
	at org.apache.spark.streaming.kafka.KafkaUtilsPythonHelper.createDirectStreamWithoutMessageHandler(KafkaUtils.scala:688)
	at sun.reflect.NativeMethodAccessorImpl.invoke0(Native Method)
	at sun.reflect.NativeMethodAccessorImpl.invoke(Nat

This is called Direct approach as it does not involve any receivers unlike the previous approach. This approach ensures stronger end-to-end guarantees. Instead of using receivers to receive data, this approach periodically queries Kafka to get the latest offsets in each topic+partition. Accordingly it defines the offset ranges to process in each batch of messages. When the jobs to process the data are launched, Kafka’s simple consumer API is used to read the defined ranges of offsets from Kafka (similar to read files from a file system).

The data is now handled as a normal RDD in our application to perform word count.

In [ ]:
lines = kafkaStream.map(lambda x: x[1])

In [ ]:
counts = lines.flatMap(lambda line: line.split(' ')).map(lambda word: (word, 1)).reduceByKey(lambda a, b: a+b)

In [ ]:
counts.pprint()

In [ ]:
ssc.start()

We can terminate it by interrupting the kernel (sending Control+C)

In [ ]:
ssc.awaitTermination()

In [ ]:
#ssc.stop()